# Linear regression model for car following

This notebook was written by Evangelos Paschalidis (evangelos.paschalidis@epfl.ch) for the Decision-aid methodologies in transportation course at EPFL (http://edu.epfl.ch/coursebook/en/decision-aid-methodologies-in-transportation-CIVIL-557). 

Please contact before distributing or reusing the material below.

## Context: car following

We have a database of acceleration behaviour in car-following situations.

**Task: Specify and estimate a linear model using as explanatory variables:**

**- Speed**

**- Relative speed (Lead speed and speed differenece)**

**- Distance**

## Load packages

Before we estimate the model let's load some packages that we are going to need. When importing a package, it is common to rename it using an abbreviation.

In [1]:
import pandas as pd # for data frame processing
import numpy as np # for some statistical procedures
from scipy.optimize import minimize # optimisation routine for parameter estimation
from scipy.stats import norm # normal distribution density function
import numdifftools as nd # we use this to calculate t-ratios and p-values
import csv # we need this to store our parameters as csv

### Let's give a name to our model

In [2]:
model_name = 'car_following_model' # Name we want to give to our model (this is used when saving the output)

### Panel structure

In [3]:
panel = 0 # switch to 1 if data is panel (any other value if not panel)

### Define if we use mixing

In [4]:
mixing = 0 # switch to 1 if we apply mixing (any other value if no mixing applied)

## Load the data

Now it is time to load the data. We can do that using the piece of code below.

**Important!** Make sure the data is in the same folder with the notebook file

In [5]:
# Command to load the data
data = pd.read_table('I80_data0.txt')

# Number of observations (we need this to caclulate goodness-of-fit indices)
Nobs = data.shape[0]

## Print the data

Let's quickly print the data. Simply type *data* in the field below

(If we had opened our data set with a different name e.g. *database*

In [6]:
# Type "data" in this field (without the quotation) and run the cell (Shift + return)
data

,ID,Time,Position,Length,Width,Type,Speed,Acceleration,Lane,Leader,...,lead04,lane01,lane02,lane03,lane04,tasks,task_index,timelag,dd,idd
0,4,26,83.033604,4.084802,-9999.99,2,6.847780,-2.266411,5,21,...,1,1,1,1,1,28,5,25,1,4
1,4,27,87.759061,4.084802,-9999.99,2,3.295830,-1.349331,5,21,...,1,1,1,1,1,28,6,26,1,4
2,4,28,90.960723,4.084802,-9999.99,2,2.820909,-2.040139,5,21,...,1,1,1,1,1,28,7,27,1,4
3,4,29,92.047244,4.084802,-9999.99,2,0.276981,0.345661,5,21,...,1,1,1,1,1,28,8,28,1,4
4,4,30,92.601824,4.084802,-9999.99,2,0.634731,0.365870,5,21,...,1,1,1,1,1,28,9,29,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18421,3340,110,397.345232,4.359100,-9999.99,2,10.053292,-0.356570,2,117,...,1,1,1,1,1,55,51,109,1,3340
18422,3340,111,406.291822,4.359100,-9999.99,2,8.161121,-0.587411,2,117,...,1,1,1,1,1,55,52,110,1,3340
18423,3340,112,414.387694,4.359100,-9999.99,2,8.106680,0.110621,2,117,...,1,1,1,1,1,55,53,111,1,3340
18424,3340,113,422.633135,4.359100,-9999.99,2,8.298771,-0.182639,2,117,...,1,1,1,1,1,55,54,112,1,3340


## Print the variable names (columns)

We can also print the variable names only using the piece of code below

* This is useful during model specification to easily access the column names

In [7]:
print(data.columns)

Index(['ID', 'Time', 'Position', 'Length', 'Width', 'Type', 'Speed',
       'Acceleration', 'Lane', 'Leader', 'Follower', 'Space_headway',
       'Time_headway', 'Acceleration_lead', 'Speed_lead', 'Position_lead',
       'Density', 'distanceToEnd', 'lane_change', 'lead_change', 'lag_leader0',
       'lag_leader1', 'lag_leader2', 'lag_leader3', 'lag_leader4', 'lag_lane0',
       'lag_lane1', 'lag_lane2', 'lag_lane3', 'lag_lane4', 'lag_acceleration0',
       'lag_acceleration1', 'lag_acceleration2', 'lag_acceleration3',
       'lag_acceleration4', 'lag_acceleration_lead0', 'lag_acceleration_lead1',
       'lag_acceleration_lead2', 'lag_acceleration_lead3',
       'lag_acceleration_lead4', 'lag_speed0', 'lag_speed1', 'lag_speed2',
       'lag_speed3', 'lag_speed4', 'lag_speed_lead0', 'lag_speed_lead1',
       'lag_speed_lead2', 'lag_speed_lead3', 'lag_speed_lead4',
       'lag_position_lead0', 'lag_position_lead1', 'lag_position_lead2',
       'lag_position_lead3', 'lag_position_lead4', '

## Variable definition

We need to define the variables (as numpy arrays) that we will use in our model.

* The arrays can have any name but it is more convenient to use the same name as in the data set.

In [72]:
# Example:
# Variable_name = np.array(data['Variable_name']).reshape(-1, 1)
# assume reaction time is 1s
Acceleration = np.array(data['Acceleration']).reshape(-1, 1)
Speed = np.array(data['Speed']).reshape(-1, 1)
Space_headway = np.array(data['lag_s_headway1']).reshape(-1, 1)
Relative_speed = np.array(data['lag_speed_lead1']- data['lag_speed1']).reshape(-1, 1)

print(Acceleration)

[[-2.2664105 ]
 [-1.34933131]
 [-2.04013918]
 ...
 [ 0.11062106]
 [-0.18263921]
 [-0.24633022]]


##### Define the ID variable

In [16]:
ID = np.array(data['ID']) # ID does not need to be reshaped

## Model specification

We now need to create a function that includes our model.

* Python functions are defined as: def *function_name*(parameters):
* We end a function as: return *value_to_return*

In the current implementation we specify two different functions as:
* *function 1* calculates the log likelihood per observations
* *function 2* calculates the sum of log likelihood taking as input the result from *function 1*

*We define two different functions to be more flexible in the post estimation processing later in the code*

We use some python (numpy) functions such '*exp*' or '*log*'. To execute these in the current example, we need to also call numpy; hence, we have *np.exp()* and *np.log()*.

### Define parameters and starting values

Ultimately, we want to estimate the value of some parameters that maximise the likelihood of our observations of the dependent variable.

Before starting the estimation process, we need to define some starting values for the parameters to be estimated.

* The starting values are usually zeroes
* When a parameter is included as a denominator, the starting value cannot be 0 for computational reasons.
* However, since we estimate the log of sigma, our starting value can be zero since exp(sigma) can never be absolute zero.

In [21]:
betas_start = {"alpha_acc": 0, "alpha_dec": 0, "beta_acc": 0, 
               "beta_dec": 0, "gamma_acc": 0, "gamma_dec": 0,
               "lamda_acc": 0, "lamda_dec": 0, "sigma_acc": 0, "sigma_dec": 0}

### Load old parameter estimates results

Sometimes, we want to use results from old models as starting values.
* To do that, we will load the iteration file from a previous estimation
* Please note that only values of parameters with same names with our current model will be copied

In [13]:
# ### Activate this cell to load old results ###

# # Open old iteration file
# betas_old = pd.read_csv('model_name_iterations.csv')

# # Keep last row
# betas_old = betas_old.tail(1)

# # Convert to dictionary
# betas_old = betas_old.to_dict(orient='records')[0]

# # Copy values from old to start for keys that are common to both dictionaries
# betas_start = {k: betas_old[k] for k in betas_start.keys() & betas_old.keys()}

# # Delete old estimates
# del[betas_old]

### Function 1: log likelihood (LL)
This function calculates the log likelihood per observation

In [81]:
def LL(betas): # betas is a vector with the parameters we want to estimate
   
    # First let's define the parameters to be estimated.
    # The parameter names are imported directly from 'beta_start' that we defined earlier
    
    for pn in range(len(betas_start.values())):
        globals()[np.array(list(betas_start.keys()))[pn]] = betas[pn]

    # Then we need to define the main model specification
    ai = (alpha_acc * ((Speed+np.exp(-50))**beta_acc)/(Space_headway**gamma_acc)) * (abs(Relative_speed)**lamda_acc)
    bi = (alpha_dec * ((Speed+np.exp(-50))**beta_dec)/(Space_headway**gamma_dec)) * (abs(Relative_speed)**lamda_dec)

    #######################################################################################
    #######################################################################################
    
    # TASK ALERT #1!! Add proximity to public transportation as independent variable
    
    # TASK ALERT #2!! Add non-linearities for in the form of b1*x + b2*(x**2)
    # Add these non-linearities for proximity to public transport, number of vehicles and 
    # household size

    #######################################################################################
    #######################################################################################
    
    # Then we need to use the probability density function of a normal distribution
    # We call the normal density function as norm.pdf(a,b,c) with inputs:
        # a: is the dependent variable (Number of trips)
        # b: this is our model (fi) which serves as the mean of the normal distribution
        # c: this is a parameter that captures the standard deviation of the error term
        
    # We implement the norm.pdf() as follows:
        # Trips: is the number of trips observed in the data
        # fi: is the model function that we defined earlier
        # sigma: is the standard deviation of the error term.
        # Instead of sigma, we estimate the log of sigma (i.e. we take the exponent) to ensure positivity of this term:
        # 1 # During estimation the sigma value may go to 0
        # 2 # If this happens, estimation may fail.
    Pa = norm.pdf(Acceleration, ai, np.exp(sigma_acc))
    Pb = norm.pdf(Acceleration, bi, np.exp(sigma_dec))
    P = Pa**(0+(Relative_speed>=0)) * Pb**(1-(Relative_speed>=0))
    # print(P)
    
    
    ############################################################################################################
    ############################################################################################################
    # - Now this below is relevant if we have panel data and apply mixing (Do not change this piece of code!) -#
    if panel == 1:
    # Do it as panel
        P = pd.DataFrame(P)
        P = pd.concat([pd.Series(ID), pd.DataFrame(P)], axis=1, ignore_index=True)
        P.rename(columns={P.columns[0]: 'ID'},inplace=True)
    
        # We take the product of probabilities per individual per draw and then delete the ID column
        P = P.groupby('ID', as_index=False).prod()
        P = P.drop('ID', axis=1)
   
    if mixing == 1:
        # We take the average per row to get the average probability per individual (if mixing == 1)
        
        if panel == 1:
            P['mean'] = P.mean(axis=1)
            P = np.array(P['mean'])
        
        if panel == 0:
            P = pd.DataFrame(P)
            P = pd.concat([pd.Series(ID), pd.DataFrame(P)], axis=1, ignore_index=True)
            P.rename(columns={P.columns[0]: 'ID'},inplace=True)
    
            # We take the product of probabilities per individual per draw and then delete the ID column
            P = P.groupby('ID', as_index=False).prod()
            P = P.drop('ID', axis=1)
            P['mean'] = P.mean(axis=1)
            P = np.array(P['mean'])
            
    P = np.array(P)

    ### --- This is where the panel data approach ends. --- ###
    ############################################################################################################
    ############################################################################################################
    
    # We then take the log of the density function
    logprob = np.log(P)
    
    return logprob

### Function 2: sum of log likelihood (SLL)
This function simply takes the sum of log likelihood that we calculated with the first function

In [82]:
def SLL(betas):
    return -sum(LL(betas))

## Model estimation

### Warnings

Sometimes, optimisation procedures may 'overdo' it with warnings during estimation.
We can supress these with the piece of code below (not always advisable)

In [83]:
import warnings
from scipy.stats import t
# with warnings.catch_warnings():
warnings.filterwarnings("ignore")

### Estimation

Now it is finally time to run our estimation command.
We use an optimisation algorith called 'BFGS'.

**It is advisable not to edit the lines of code below**

In [84]:
# This will give us the initial loglikelihood value as an output
def callback1(betas):
    print("Current log likelihood:", -SLL(betas))

# This function will allow as to store parameter estimates during iterations
# Initialise list to store parameter values
parameter_values = [np.array(list(betas_start.values()))]
# Then define the function
def callback2(betas):    
    parameter_values.append(betas)
    column_names = list(betas_start.keys())
    with open(f'{model_name}_iterations.csv','w',newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        writer.writerows(parameter_values)

# Now let's combine the two callback functions
def combined_callback(betas):
    callback1(betas)
    callback2(betas)
        
print("Initial log likelihood:", -SLL(np.array(list(betas_start.values()))))

# Choose optimisation routine (preferably BFGS)
optimiser = 'BFGS' # BFGS or L-BFGS-B or nelder-mead

result = minimize(SLL, np.array(list(betas_start.values())), method=optimiser,callback=combined_callback, 
                  options={'disp':False}) # ,bounds=bounds1
#args = (parameter_values,)
print("Final log likelihood:", -result.fun)

Initial log likelihood: [-24996.76945136]
Current log likelihood: [-23306.25770674]
Current log likelihood: [-23152.88282405]
Current log likelihood: [-22944.0623558]
Current log likelihood: [-22786.13969629]
Current log likelihood: [-22638.30327478]
Current log likelihood: [-22618.96981057]
Current log likelihood: [-22592.6731065]
Current log likelihood: [-22482.28361215]
Current log likelihood: [-22462.72309787]
Current log likelihood: [-22462.19954986]
Current log likelihood: [-22441.21291372]
Current log likelihood: [-22402.57731738]
Current log likelihood: [-22340.85645572]
Current log likelihood: [-22318.87079694]
Current log likelihood: [-22314.46872735]
Current log likelihood: [-22307.8379214]
Current log likelihood: [-22303.06070007]
Current log likelihood: [-22301.69367274]
Current log likelihood: [-22299.7089304]
Current log likelihood: [-22297.70801381]
Current log likelihood: [-22295.31739583]
Current log likelihood: [-22293.0226998]
Current log likelihood: [-22291.4552078

## Post estimation processing

We evaluate our parameter estimates using t-ratios (or p-Values).

In maximum likelihood estimation, we extract these from the variance-covariance matrix of the parameters.

The variance covariance matrix is not readily available but we need to calculate it.

This is done with the code below.

**DO NOT EDIT THE CHUNK OF CODE BELOW!**

In [85]:
# Vector of parameter estimates
parameters = result['x'] 

# Calculate hessian
print("Calculating Hessian, please wait (this may take a while...)")
Hess = nd.Hessian(SLL, method = 'forward')
hessian = Hess(parameters)
inv_hessian = np.linalg.inv(hessian)

# Parameter statistics
dof = Nobs - len(betas_start) - 1
se = np.sqrt(np.diag(inv_hessian)) # Standard errors
tratio = parameters/se # t-ratios
p_value = (1-t.cdf(np.abs(tratio),dof)) * 2 # p-values


# --- Sandwich estimator --- #

# The sandwich estimator provides the "robust" s.e., t-ratios and p-values.
# These should be preferred over the classical parameter statistics.

# We first need the gradients at the solution point
Grad = nd.Gradient(LL, method = 'forward')
gradient = Grad(parameters)

# Then we need to calculate the B matrix
B = np.array([])
for r in range(gradient.shape[0]):
    Bm = np.zeros([len(betas_start),len(betas_start)])
    gradient0 = gradient[r,:]
    for i in range(len(gradient0)):
            for j in range(len(gradient0)):
                element = gradient0[i]*gradient0[j]
                Bm[i][j] = element
    if B.size==0:
                    B = Bm
    else:
                    B = B+Bm

# Finally we "sandwich" the B matrix between the inverese hessian matrices
BHHH = (inv_hessian)@(B)@(inv_hessian)

print("... Done!!")

# Now it is time to calculate some "robust" parameter statistics
rob_se = np.sqrt(np.diag(BHHH)) # robust standard error
rob_tratio = parameters/rob_se # robust t-ratio
rob_p_value = (1-t.cdf(np.abs(rob_tratio),dof)) * 2 # robust p-value

Calculating Hessian, please wait (this may take a while...)
... Done!!


## Results
Finally, we got our results. Let's print them!

The outputs that we receive are:
* Estimates: These are the values of our parameters. We must check if the sign is consistent with our expectations
* s.e.: Standard errors of the parameters
* tratio: t-ratio of the parameters (significant if absolute value > 1.96)
* p_value: p-value of the parameters (significant if < 0.05)

The parameter statistics also have their **robust** versions. These should be preferred as they are less susceptible to mis-specified models.

In [86]:
arrays = np.column_stack((np.array(list(betas_start.keys())),parameters,se,tratio,p_value,rob_se,rob_tratio,rob_p_value))
results = pd.DataFrame(arrays, columns = ['Parameter','Estimate','s.e.','t-ratio0','p-value',
                                          'Rob s.e.','Rob t-ratio0','Rob p-value'])

results[['Estimate','s.e.','t-ratio0','p-value','Rob s.e.','Rob t-ratio0','Rob p-value']] = (
results[['Estimate','s.e.','t-ratio0','p-value','Rob s.e.','Rob t-ratio0','Rob p-value']].apply(pd.to_numeric, errors='coerce'))
numeric_cols = results.select_dtypes(include='number').columns
results[numeric_cols] = results[numeric_cols].round(3)
results # print results

,Parameter,Estimate,s.e.,t-ratio0,p-value,Rob s.e.,Rob t-ratio0,Rob p-value
0,alpha_acc,2.036,0.193,10.526,0.000,0.231,8.798,0.000
1,alpha_dec,-2.717,0.274,-9.911,0.000,0.356,-7.634,0.000
2,beta_acc,0.304,0.040,7.640,0.000,0.045,6.712,0.000
3,beta_dec,0.081,0.036,2.233,0.026,0.044,1.832,0.067
4,gamma_acc,0.719,0.047,15.310,0.000,0.054,13.288,0.000
5,gamma_dec,0.710,0.049,14.386,0.000,0.064,11.056,0.000
6,lamda_acc,0.904,0.030,30.128,0.000,0.033,27.702,0.000
7,lamda_dec,1.006,0.027,37.773,0.000,0.031,31.978,0.000
8,sigma_acc,-0.263,0.007,-36.131,0.000,0.009,-28.766,0.000
9,sigma_dec,-0.230,0.007,-30.773,0.000,0.010,-23.060,0.000


## Goodness-of-fit indices

Let's calculate some goodness-of-fit indices now (do not edit)

In [ ]:
# First let's calculate the GoF indices

rho_squared = 1 - ((-result.fun)/(-SLL(np.zeros(len(betas_start)))))
adj_rho_squared = 1 - (((-result.fun)-len(betas_start))/(-SLL(np.zeros(len(betas_start)))))

AIC = 2*len(betas_start) - 2*(-result.fun)
BIC = len(betas_start)*np.log(Nobs) - 2*(-result.fun)

LL0t = "Log likelihood at zeros:" + str(-SLL(np.zeros(len(betas_start))))
LLinit = "Initial log likelihood:" + str(-SLL(np.array(list(betas_start.values()))))
LLfin = "Final log likelihood:" + str(-result.fun)

rs1 = "rho squared="+str(rho_squared)
rs2 = "adjusted rho squared="+str(adj_rho_squared)
ac = "AIC="+str(AIC)
bc = "BIC="+str(BIC)

# Then let's print the GoF indices

print(LL0t)
print(LLinit)
print(LLfin)

print(rs1)
print(rs2)
print(ac)
print(bc)

## Save output

We can save our output using the code below (do not edit):

In [ ]:
with open(f"{model_name}_results.txt", 'w') as f:
    f.write(f'{LL0t}\n')
    f.write(f'{LLinit}\n')
    f.write(f'{LLfin}\n')
    f.write(f'{rs1}\n')
    f.write(f'{rs2}\n')
    f.write(f'{ac}\n')
    f.write(f'{bc}\n')
    results.to_csv(f, index=False, sep='\t')
results.to_csv(f'{model_name}_results.csv', index=False)